In [8]:
import pandas as pd

In [9]:
df_skaters = pd.read_csv("skaters_1996_2023.csv")
df_goalies = pd.read_csv("goalies_1996_2023.csv")

In [10]:
df_skaters.query('PLAYER == "Nicholas Abruzzese"')

,RK,PLAYER,AGE,TM,POS,SCORING_GP,SCORING_G,SCORING_A,SCORING_PTS,SCORING_+/-,...,SHOT_DATA_S,SHOT_DATA_S%,ICE_TIME_TOI,ICE_TIME_ATOI,BLK,HIT,FOW,FOL,FO%,SEASON
0,1,Nicholas Abruzzese,23,TOR,C,2,0,2,2,1,...,2,0.0,18,8:59,0,1,0,0,NaN,2023
952,1,Nicholas Abruzzese,22,TOR,C,9,1,0,1,-1,...,8,12.5,92,10:13,3,7,2,5,28.6,2022


In [6]:
# Define the weights for the past three seasons
weights = {1: 0.5, 2: 0.3, 3: 0.2}

# Define the age adjustment factors (hypothetical)
age_adjustment = {i: 1 - (i - 28) * 0.01 for i in range(18, 50)}

def calculate_weighted_ps(player_data: pd.DataFrame):
    # Calculate weighted point shares based on past three seasons
    weighted_ps = sum(weights[i] * player_data['PS'].shift(i) for i in weights)
    return weighted_ps

def apply_age_adjustment(player_data: pd.DataFrame):
    # Apply age adjustment factor
    player_data['FORECAST_PS'] *= age_adjustment.get(player_data['AGE'], 1)
    return player_data

def apply_regression(player_data: pd.DataFrame):
    # Apply regression towards the mean
    reliability = min(1, len(player_data) / 1000)
    league_average_ps = df_skaters['PS'].mean()
    player_data['FORECAST_PS'] = reliability * player_data['FORECAST_PS'] + (1 - reliability) * league_average_ps
    return player_data

# Sort the data by season
df_skaters.sort_values('SEASON', inplace=True)

# Group by player and apply the forecasting system
df_skaters['FORECAST_PS'] = df_skaters.groupby('PLAYER').apply(calculate_weighted_ps)
df_skaters = df_skaters.groupby('PLAYER').apply(apply_age_adjustment)
df_skaters = df_skaters.groupby('PLAYER').apply(apply_regression)



TypeError: can't multiply sequence by non-int of type 'float'